In [ ]:

!pip install finance-datareader
!pip install pycaret

     |████████████████████████████████| 288 kB 7.2 MB/s 
     |████████████████████████████████| 167 kB 59.5 MB/s 
     |████████████████████████████████| 6.8 MB 52.5 MB/s 
     |████████████████████████████████| 113 kB 60.7 MB/s 
     |████████████████████████████████| 86 kB 5.1 MB/s 
     |████████████████████████████████| 1.7 MB 51.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.0 MB 53.1 MB/s 
     |████████████████████████████████| 261 kB 44.1 MB/s 
     |████████████████████████████████| 56 kB 3.8 MB/s 
     |████████████████████████████████| 16.9 MB 45.0 MB/s 
     |████████████████████████████████| 1.3 MB 38.0 MB/s 
     |████████████████████████████████| 303 kB 50.5 MB/s 
     |████████████████████████████████| 62 kB 810 kB/s 
     |████████████████████████████████| 10.1 MB 54.3 MB/s 
     |██████████████████

In [ ]:
import pandas as pd
import numpy as np
import os
from pycaret.regression import *
import FinanceDataReader as fdr
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from tqdm import tqdm

## Get Stock List

In [ ]:
path = ''
list_name = 'stock_list.csv'
sample_name = 'sample_submission.csv'
stock_data = 'stock_data_until21_11_19(11-23).csv'
merge_stock_data = '20210104-20211119_stock.csv'

stock_list = pd.read_csv(os.path.join(path,list_name))
stock_data = pd.read_csv(os.path.join(path,stock_data))
merge_stock_data = pd.read_csv(os.path.join(path,merge_stock_data))
stock_data['Date'] = pd.to_datetime(stock_data['Date'])
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))


In [ ]:
start_date = '20210104'
end_date = '20211119'

In [ ]:
# import lightgbm as lgb
# model = LinearRegression()
# for code in tqdm(stock_list['종목코드'].values):
#   data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
#   data = pd.merge(stock_data, data,on='Date',how='left').set_index('Date')
#   X = data[fratures]
#   y = data['Close']
#   X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
# Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])
# code = '005930'
# data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
# data = pd.merge(Business_days, data, how = 'outer')
# data['weekday'] = data.Date.apply(lambda x : x.weekday())
# data['weeknum'] = data.Date.apply(lambda x : x.strftime('%V'))
# data.Close = data.Close.interpolate()


In [ ]:
# merge_stock_data.to_datetime()
# data = pd.merge(data, merge_stock_data, on='Date')

In [ ]:
sample_code = stock_list.loc[0,'종목코드']

sample = fdr.DataReader(sample_code, start = start_date, end = end_date)[['Close']].reset_index()

sample = pd.merge(stock_data,sample,on='Date',how='left')

train = sample[sample['Date'] < '2021-11-01']
test =  sample[sample['Date'] <= '2021-11-05']
test =  test[test['Date'] >= '2021-11-01']
test = pd.DataFrame(test['Date']).reset_index().drop('index',axis=1)
test

,Date
0,2021-11-01
1,2021-11-02
2,2021-11-03
3,2021-11-04
4,2021-11-05


In [ ]:

    
features = ['Date','NASDAQ','BTC', 'US2YT','US10YT','Gold','Oil','Close']
train = train[features]
stock_pred_dict = {} 

  
for code in tqdm(stock_list['종목코드'].values):
  sample_code = stock_list.loc[code,'종목코드']
  sample = fdr.DataReader(sample_code, start = start_date, end = end_date)[['Close']].reset_index()
  sample = pd.merge(stock_data,sample,on='Date',how='left')

  train = sample[sample['Date'] < '2021-11-01']
  test =  sample[sample['Date'] <= '2021-11-05']
  test =  test[test['Date'] >= '2021-11-01']
  for i in range(7):
    model=setup(
        data=train,
        target = features[i+1],
        ignore_features = features[i+2:],
        log_experiment = True,
        use_gpu = True
      )
    tuned_model = tune_model(compare_models(sort='MAPE', n_select=1))
    tuned_model_ = finalize_model(tuned_model)
    pred = predict_model(tuned_model_, test)
    test[features[i+1]] = pred['Label']

  test = pd.DataFrame(test['Date']).reset_index().drop('index',axis=1)
    
  stock_pred_dict[code] = test['Close']


,Description,Value
0,session_id,6633
1,Target,US2YT
2,Original Data,"(215, 8)"
3,Missing Values,True
4,Numeric Features,2
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(142, 19)"


IntProgress(value=0, description='Processing: ', max=94)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,21:30:29
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Extra Trees Regressor


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,0.0153,0.0005,0.0212,0.9238,0.0173,0.0772,0.873
dt,Decision Tree Regressor,0.0197,0.0009,0.0289,0.8535,0.0233,0.0986,0.010
lar,Least Angle Regression,0.0232,0.0010,0.0308,0.8326,0.0247,0.1159,0.017
br,Bayesian Ridge,0.0238,0.0010,0.0310,0.8313,0.0248,0.1178,0.009
ridge,Ridge Regression,0.0247,0.0010,0.0315,0.8270,0.0253,0.1243,0.021
lasso,Lasso Regression,0.0425,0.0030,0.0541,0.4941,0.0429,0.2108,0.010
en,Elastic Net,0.0426,0.0030,0.0541,0.4912,0.0430,0.2115,0.010
lr,Linear Regression,0.0427,0.0030,0.0541,0.4882,0.0430,0.2124,0.009
omp,Orthogonal Matching Pursuit,0.0406,0.0023,0.0468,0.5765,0.0384,0.2226,0.013
knn,K Neighbors Regressor,0.0506,0.0048,0.0666,0.1973,0.0539,0.2712,0.030


NameError: ignored

Lars(copy_X=True, eps=0.007, fit_intercept=True, fit_path=True, jitter=None,
     n_nonzero_coefs=500, normalize=True, precompute='auto', random_state=2472,
     verbose=False)

(42, 5)

- 예측

[70206.67660106532,
 69631.42785252717,
 69062.32129096359,
 69258.21096883612,
 68846.00977524316]

- 실제 Public 값

array([69900., 71500., 70400., 70600., 70200.])

100%|██████████| 370/370 [00:37<00:00,  9.78it/s]


0

Index(['Day', '000060', '000080', '000100', '000120', '000150', '000240',
       '000250', '000270', '000660',
       ...
       '330860', '336260', '336370', '347860', '348150', '348210', '352820',
       '357780', '363280', '950130'],
      dtype='object', length=371)

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,...,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-01,27919.530611,34687.673458,60773.779528,142621.815394,104901.698658,16669.447967,47219.595113,85236.833170,103490.352393,...,49749.405974,51984.322942,84384.021784,36846.592704,25721.026664,53328.350326,336697.743579,262257.538308,27176.085090,17382.219194
1,2021-11-02,28750.750484,35032.651375,60312.130021,143012.927861,107216.342323,17001.594758,46672.351191,85360.327648,102788.687368,...,48923.253693,51539.564130,84739.157977,35258.227509,25330.528060,53645.466661,335662.149461,264562.230652,27415.549191,17338.522537
2,2021-11-03,28858.095631,34995.888574,60241.470410,145626.792237,111192.733424,17665.577952,45757.683516,85665.326378,102943.419081,...,48834.428543,49341.153729,85450.806060,35320.479243,25623.131825,55136.121480,329167.718872,264623.119599,27466.446666,17286.577226
3,2021-11-04,28901.301911,34866.098057,59701.569734,145351.536595,109530.741544,17846.022961,46380.152350,85494.864447,99958.476851,...,48954.935640,48626.256108,84823.889868,34343.016848,25878.115140,55964.135511,329482.609718,261821.140588,27594.351745,17247.876124
4,2021-11-05,28152.938540,34873.939080,59968.393160,143505.827198,108761.777883,18078.266972,46975.701291,84943.135732,100294.829339,...,49427.019462,47063.105078,86397.651814,34062.808374,26472.657621,55323.587424,321108.356663,264131.897754,27408.366650,17492.773824
5,2021-11-29,27919.530611,34687.673458,60773.779528,142621.815394,104901.698658,16669.447967,47219.595113,85236.833170,103490.352393,...,49749.405974,51984.322942,84384.021784,36846.592704,25721.026664,53328.350326,336697.743579,262257.538308,27176.085090,17382.219194
6,2021-11-30,28750.750484,35032.651375,60312.130021,143012.927861,107216.342323,17001.594758,46672.351191,85360.327648,102788.687368,...,48923.253693,51539.564130,84739.157977,35258.227509,25330.528060,53645.466661,335662.149461,264562.230652,27415.549191,17338.522537
7,2021-12-01,28858.095631,34995.888574,60241.470410,145626.792237,111192.733424,17665.577952,45757.683516,85665.326378,102943.419081,...,48834.428543,49341.153729,85450.806060,35320.479243,25623.131825,55136.121480,329167.718872,264623.119599,27466.446666,17286.577226
8,2021-12-02,28901.301911,34866.098057,59701.569734,145351.536595,109530.741544,17846.022961,46380.152350,85494.864447,99958.476851,...,48954.935640,48626.256108,84823.889868,34343.016848,25878.115140,55964.135511,329482.609718,261821.140588,27594.351745,17247.876124
9,2021-12-03,28152.938540,34873.939080,59968.393160,143505.827198,108761.777883,18078.266972,46975.701291,84943.135732,100294.829339,...,49427.019462,47063.105078,86397.651814,34062.808374,26472.657621,55323.587424,321108.356663,264131.897754,27408.366650,17492.773824
